In [40]:
import astropy.units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, ICRS, CartesianRepresentation, CartesianDifferential
from astropy.time import Time

In [53]:
loc = EarthLocation(lat = 22.5726*u.deg, lon = 88.3639*u.deg, height = 0*u.m)
time = Time(1515110400, format='unix')
newAltAz = AltAz(alt = 32.42851414*u.deg, az = 192.8100706*u.deg, distance=191035815.3*u.km, obstime = time, location = loc)
newAltAz.representation = "cartesian"
newAltAz
velocity = AltAz(x = newAltAz.x, y = newAltAz.y, z = newAltAz.z, obstime = time, location = loc, v_x=31*u.km/u.s, v_y=-10*u.km/u.s, v_z=75*u.km/u.s, representation=CartesianRepresentation, differential_cls=CartesianDifferential )
obj = velocity.transform_to(ICRS)
print velocity
obj.representation = "cartesian"
obj.differential_cls = CartesianDifferential
print obj

<AltAz Coordinate (obstime=1515110400.0, location=(168237.7004225489, 5890037.320352326, 2433082.6015647054) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (x, y, z) in km
    ( -1.57232565e+08, -35751426.00598929,   1.02442368e+08)
 (v_x, v_y, v_z) in km / s
    ( 31., -10.,  75.)>